MIT License

Copyright (c) 2021  Pablo Daurell Marina,
                    Belén García Puente,
                    Ela Katherine Shepherd Arévalo,
                    Miquel Vera Ramis,
                    Alberto García Doménech,
                    Fernando Bellot Rodríguez,
                    Mateo González de Miguel


Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [1]:
import pandas as pd
import numpy as np

In [2]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  COLLAB = True
else:
  COLLAB = False

In [3]:
if COLLAB == True:
  df = pd.read_csv('/content/drive/Shareddrives/ETICA/dataset_newtral_ucm.csv')
else:
  df = pd.read_csv('../data/dataset_newtral_ucm.csv')

In [34]:
# !pip install transformers

     |████████████████████████████████| 3.4 MB 26.9 MB/s 
     |████████████████████████████████| 596 kB 47.6 MB/s 
     |████████████████████████████████| 61 kB 433 kB/s 
     |████████████████████████████████| 895 kB 48.3 MB/s 
     |████████████████████████████████| 3.3 MB 46.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [35]:
import transformers
from transformers import pipeline
pipe = pipeline("text-classification", model="Newtral/xlm-r-finetuned-toxic-political-tweets-es")

Downloading:   0%|          | 0.00/753 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/421 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [49]:
lideres = ["Casado", "Ayuso", "Abascal", "Monasterio" ,"Sanchez", "Sánchez", "Iglesias", "Yolanda", "Arrimadas", "Rivera"]
ids = ["@pablocasado_", "@IdiazAyuso", "@Santi_ABASCAL", "@monasterioR", "@sanchezcastejon", "@Yolanda_Diaz_", "@InesArrimadas"] #Pablo iglesias y Albert Rivera salen demasiado poco sus @
nombresLideres  = set(lideres).union(set(ids))
nombresLideres

{'@IdiazAyuso',
 '@InesArrimadas',
 '@Santi_ABASCAL',
 '@Yolanda_Diaz_',
 '@monasterioR',
 '@pablocasado_',
 '@sanchezcastejon',
 'Abascal',
 'Arrimadas',
 'Ayuso',
 'Casado',
 'Iglesias',
 'Monasterio',
 'Rivera',
 'Sanchez',
 'Sánchez',
 'Yolanda'}

In [50]:
df.head(5)

,twitter_id,text,toxicity,sev_toxicity,profanity,threat,degree1,degree2,degree3,degree_final,degree_predicted,degree_confidence,annotation_date,author_id,slug,twitter_url,party_slug,parliamentary_group_slug
0,1344806255798480896,Feliz 2021 #Adios2020 https://t.co/ULpRE0T2AI,0.012811,0.022777,0.010508,0.077519,0,0,0,0,0,0.969,2021-10-07,2849798849,jose-alberto-herrero-bono,https://twitter.com/herrerobono,partido-popular,gp
1,1344822586396602368,Empezar el año con #CachitosNochevieja no tien...,0.085995,0.068680,0.302721,0.087808,0,0,0,0,0,0.984,2021-10-07,88802757,begona-nasarre-oliva,https://twitter.com/Begonasarre,psoe,gs
2,1344934218145669120,Llegará un momento que los miembros de este Go...,0.478526,0.122059,0.026271,0.513052,2,2,1,2,2,0.868,2021-10-07,593952938,juan-luis-steegmann-olmedillas,https://twitter.com/jlsteeg,vox,gvox
3,1344940715072221184,La frase “ Falsa sensación de seguridad” viene...,0.309307,0.040751,0.009823,0.122409,2,1,1,1,2,0.557,2021-08-25,593952938,juan-luis-steegmann-olmedillas,https://twitter.com/jlsteeg,vox,gvox
4,1344945712128266240,El Comisario político del Gobierno no descansa...,0.194637,0.072107,0.026215,0.249154,2,2,2,2,2,0.964,2021-10-07,1197128623364423680,macarena-montesinos-de-miguel,https://twitter.com/MackMontesinos,partido-popular,gp


In [52]:
column_names = ["twitter_id", "original_text", "slug", "party_slug", "original_name", "toxic_original", "very_toxic_original", "most_toxic_leader", "most_toxic_leader_toxicity", "most_verytoxic_leader" ,"most_verytoxic_leader_toxicity", "LIDER_toxic", "LIDER_verytoxic"]
df_wordmod = pd.DataFrame(columns = column_names)
i = 0
for idx, row in df.iterrows():
  for word in row.text.split():
    if(word in nombresLideres):
      row_tox_value = pipe(row.text, return_all_scores=True)[0][0]['score']
      row_vtox_value = pipe(row.text, return_all_scores=True)[0][1]['score']
      maxt = row_tox_value
      maxtp = word
      maxvt = row_vtox_value
      maxvtp = word
      for otro in nombresLideres:
        if word != otro:
          value = pipe(row.text.replace(word, otro), return_all_scores=True)[0]
          # if pipe(row.text.replace(word, otro), return_all_scores=True)[0][0]['score'] > maxt:
          if (value[0]['score'] > maxt):
            # maxt = pipe(row.text.replace(word, otro), return_all_scores=True)[0][0]['score']
            maxt = value[0]['score']
            maxtp = otro
          # if pipe(row.text.replace(word, otro), return_all_scores=True)[0][1]['score'] > maxvt:
          if (value[1]['score'] > maxvt):
            # maxvt = pipe(row.text.replace(word, otro), return_all_scores=True)[0][1]['score']
            maxvt = value[1]['score']
            maxvtp = otro
      neutral_word = pipe(row.text.replace(word, "partido"), return_all_scores=True)[0]
      new_row = {'twitter_id':row.twitter_id, 'original_text':row.text, 'slug':row.slug, 'party_slug':row.party_slug, 'original_name': word, 'toxic_original':row_tox_value , 'very_toxic_original':row_vtox_value,'most_toxic_leader':maxtp, 'most_toxic_leader_toxicity': maxt, 'most_verytoxic_leader':maxvtp, 'most_verytoxic_leader_toxicity': maxvt, 'LIDER_toxic': neutral_word[0]['score'],'LIDER_verytoxic':neutral_word[1]['score']}
      df_wordmod = df_wordmod.append(new_row, ignore_index = True)
      i+=1

In [53]:
df_wordmod

,twitter_id,original_text,slug,party_slug,original_name,toxic_original,very_toxic_original,most_toxic_leader,most_toxic_leader_toxicity,most_verytoxic_leader,most_verytoxic_leader_toxicity,LIDER_toxic,LIDER_verytoxic
0,1345036097030643712,Un PP unido y dirigiéndose al liderazgo necesa...,andres-lorite-lorite,partido-popular,@pablocasado_,0.004291,0.002394,Iglesias,0.005631,Arrimadas,0.002432,0.004231,0.002415
1,1345429153357701121,El acuerdo del Gobierno con Reino Unido sobre ...,concepcion-gamarra-ruiz-clavijo,partido-popular,Sánchez,0.004959,0.002321,@Santi_ABASCAL,0.005231,@monasterioR,0.002508,0.006789,0.001784
2,1345630564498481153,Los españoles hartos de la irresponsabilidad d...,carmen-riolobos-regadera,partido-popular,Iglesias,0.993780,0.109731,Monasterio,0.994659,Monasterio,0.188398,0.993590,0.099156
3,1345630564498481153,Los españoles hartos de la irresponsabilidad d...,carmen-riolobos-regadera,partido-popular,@pablocasado_,0.993780,0.109731,Sánchez,0.996026,Sánchez,0.474128,0.995812,0.297186
4,1345650372191379456,Casado se sitúa a menos de tres puntos de Sánc...,ana-belen-vazquez-blanco,partido-popular,Casado,0.032812,0.000795,Monasterio,0.236452,Ayuso,0.001158,0.060692,0.000806
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1682,1453339175256743940,La vivienda es sobre todo un derecho humano y ...,alfonso-rodriguez-gomez-de-celis,psoe,@sanchezcastejon,0.004295,0.002777,Iglesias,0.010003,Abascal,0.003100,0.004707,0.002406
1683,1453238212076392448,El cambio en España no hay quien lo pare …\n\n...,jose-ortiz-galvan,partido-popular,@pablocasado_,0.010026,0.001155,Abascal,0.242209,Yolanda,0.001339,0.043656,0.000735
1684,1451049912062386184,67 días desde que empezó el #IFNavalacruz en #...,alicia-garcia-rodriguez,partido-popular,Sánchez,0.006563,0.002006,Iglesias,0.008175,@IdiazAyuso,0.002317,0.010167,0.001447
1685,1452567812501905410,Los españoles lo tienen claro: @pablocasado_ e...,NaN,NaN,@pablocasado_,0.060123,0.000780,Ayuso,0.321642,Yolanda,0.001149,0.065662,0.000897


In [54]:
if COLLAB == True:
  df_wordmod.to_csv('/content/drive/Shareddrives/ETICA/tweets_mods_politicians.csv', index=False)  
else:
  df_wordmod.to_csv('../data/tweets_mods_politicians.csv', index=False)